In [1]:
import pandas as pd
import glob
import os
from paths import *

In [2]:
# Function to concatenate native language columns
def concatenate_native_languages(row):
    native_language_cols = ['Q5_1_TEXT', 'Q6_1', 'Q6_2', 'Q6_3', 'Q6_4', 'Q6_5']
    native_languages = [str(row[col]) for col in native_language_cols if not pd.isna(row[col])]
    return ','.join(native_languages)

In [3]:
def get_data(df): 
    # Filter out rows where the "task" column is empty
    df = df[df['task'].notna()]
    # extract the desired columns
    cols = ['participant', 'token_id', 'task', 'continuum', 'vowel', 'series', 'sound_1', 'sound_2', 'img', 'same', 'trained', 'FJ_attn.keys', 'FJ_attn.rt', 'FJ_test.keys', 'FJ_test.rt']
    df = df[cols]

    # merge the FJ_attn and FJ_test columns
    df['FJ_ans'] = df['FJ_attn.keys'].fillna(df['FJ_test.keys'])
    df['FJ_rt'] = df['FJ_attn.rt'].fillna(df['FJ_test.rt'])
    df.drop(['FJ_attn.keys', 'FJ_attn.rt', 'FJ_test.keys', 'FJ_test.rt'], axis=1, inplace=True)
    
    # Add an extra index called "exp_order"
    df['exp_order'] = df.index - df.index[0] + 1
    return df

In [4]:
# get_data(pd.read_csv(data_dir + "5a0c036857cee70001767a58_phx_ver10_2023-06-16_10h19.28.157.csv"))

In [5]:
# Read the csv files
prolific = pd.read_csv(prolific_path)
qualtrics = pd.read_csv(qualtrics_path)

# Rename the column names in prolific dataframe
prolific.rename(columns={'Participant id': 'PROLIFIC_PID',
                         'Age': 'age_prolific',
                         'Sex': 'sex_prolific',
                         'Nationality': 'nationality_prolific',
                         'Language': 'language_prolific'}, inplace=True)
# Keep only the approved entries in prolific
prolific = prolific[prolific['Status'] == 'APPROVED']

# Merge the prolific and qualtrics dataframes on PROLIFIC_PID
merged = pd.merge(prolific, qualtrics, on='PROLIFIC_PID', how='left')

# Add 'native_languages_qual' column
merged['native_languages_qual'] = merged.apply(concatenate_native_languages, axis=1)

# Rename the 'Q7_1_TEXT' column
merged.rename(columns={'Q1': 'gender_qual', 
                       'Q3': 'age_qual', 
                       'Q7_1_TEXT': 'other_languages_qual'}, inplace=True)

# Get all csv files in the "data" subdirectory
data_files = glob.glob(data_dir + '*_phx_ver10_*.csv')

keep_cols = ['PROLIFIC_PID', "age_prolific", "age_qual","sex_prolific", "gender_qual", "nationality_prolific", "language_prolific", "native_languages_qual", "other_languages_qual"]

merged = merged[keep_cols]

# Rename PROLIFIC_PID to participant
merged.rename(columns={'PROLIFIC_PID': 'participant'}, inplace=True)

# Initialize an empty dataframe to store the final merged data
final_merged_data = pd.DataFrame()

# Iterate through the data files
for data_file in data_files:
    # Read the csv file
    data = pd.read_csv(data_file)

    data = get_data(data)

    # Extract the participant id from the file name
    participant_id = data_file.split('_')[0].split('/')[-1]

    # Merge the data with the merged dataframe based on the participant id
    merged_data = pd.merge(merged[merged['participant'] == participant_id], data, on='participant', how='left')

    # Append the merged_data to the final_merged_data dataframe
    final_merged_data = pd.concat((final_merged_data, merged_data))

# Save the final merged data to a single csv file
final_merged_data.to_csv(out_path, index=False)